In [ ]:
# Install required libraries (uncomment if running in Colab)
%pip install --upgrade torch transformers datasets pandas --quiet
%pip install "transformers[torch]" "accelerate>=0.26.0"
%pip install matplotlib seaborn
%pip install protobuf
%pip install rouge-score matplotlib-venn nltk
%pip install wordcloud

In [ ]:
import os
import pandas as pd
from datasets import Dataset
import zipfile

# File paths for the downloaded files (update with actual paths if needed)
train_zip_path = "/Users/sajinshrestha/Developer/college/text_summarizer/datasets/train.csv.zip"
val_csv_path = "/Users/sajinshrestha/Developer/college/text_summarizer/datasets/val.csv"
test_csv_path = "/Users/sajinshrestha/Developer/college/text_summarizer/datasets/test.csv"

# Directory to extract train.csv from train.csv.zip
extract_dir = "./extracted_datasets"
os.makedirs(extract_dir, exist_ok=True)

# Unzip train.csv.zip
with zipfile.ZipFile(train_zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)
print(f"Extracted train.csv.zip to {extract_dir}")

# Find the extracted train.csv
extracted_files = os.listdir(extract_dir)
train_csv_path = None
for file in extracted_files:
    if file.lower() == 'train.csv':
        train_csv_path = os.path.join(extract_dir, file)
        break
if train_csv_path is None:
    raise FileNotFoundError(f"train.csv not found in {extract_dir}. Extracted files: {extracted_files}")

# Load CSVs using pandas to avoid filesystem issues
try:
    train_df = pd.read_csv(train_csv_path)
    validation_df = pd.read_csv(val_csv_path)
    test_df = pd.read_csv(test_csv_path)
except Exception as e:
    print(f"Error loading CSV files: {e}")
    raise

# Convert pandas DataFrames to HuggingFace Datasets
train_dataset = Dataset.from_pandas(train_df)
validation_dataset = Dataset.from_pandas(validation_df)
test_dataset = Dataset.from_pandas(test_df)

# Print column names for each dataset
print("\nColumn names in Train Dataset:", train_dataset.column_names)
print("Column names in Validation Dataset:", validation_dataset.column_names)
print("Column names in Test Dataset:", test_dataset.column_names)

# Preview the datasets
print(f"\nTrain data (total {len(train_dataset)} rows)")
print(f"\nValidation data (total {len(validation_dataset)} rows)")
print(f"\nTest data (total {len(test_dataset)} rows)")

In [ ]:
#%pip install "transformers[torch]" "accelerate>=0.26.0"

import pandas as pd
from datasets import Dataset
import zipfile
import os
import re
from transformers import (
    BartTokenizer,
    BartForConditionalGeneration,
    TrainingArguments,
    Trainer,
    DataCollatorForSeq2Seq,
    EarlyStoppingCallback
)

# --- Data Cleaning Function ---
def clean_text(text):
    text = re.sub(r'<.*?>', '', text)  # Remove HTML tags
    text = re.sub(r'\s+', ' ', text)   # Normalize whitespace
    text = re.sub(r'[^a-zA-Z0-9\s.,!?]', '', text)  # Keep alphanumeric and select punctuation
    return text.strip()

# --- Load and Prepare Datasets ---
extract_dir = "/content/extracted_datasets"
os.makedirs(extract_dir, exist_ok=True)
with zipfile.ZipFile("/content/train.csv.zip", 'r') as zip_ref:
    zip_ref.extractall(extract_dir)
train_csv_path = os.path.join(extract_dir, 'train.csv')

train_df = pd.read_csv(train_csv_path).dropna(subset=['document', 'summary']).astype(str)
validation_df = pd.read_csv("/content/val.csv").dropna(subset=['document', 'summary']).astype(str)

# Clean and filter data
for df in [train_df, validation_df]:
    df['document'] = df['document'].apply(clean_text)
    df['summary'] = df['summary'].apply(clean_text)
    df.drop(df[df['document'].str.len() < 50].index, inplace=True)
    df.drop(df[df['summary'].str.len() < 10].index, inplace=True)

# Load tokenizer for filtering
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large-cnn')

# Filter by token length
train_df = train_df[train_df['summary'].apply(lambda x: 15 < len(tokenizer(x, add_special_tokens=True)['input_ids']) <= 120)]
validation_df = validation_df[validation_df['summary'].apply(lambda x: 15 < len(tokenizer(x, add_special_tokens=True)['input_ids']) <= 120)]

# Shuffle training data
train_df = train_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Convert to HuggingFace Datasets
train_dataset = Dataset.from_pandas(train_df)
validation_dataset = Dataset.from_pandas(validation_df)

# --- Preprocessing function ---
def preprocess_function(examples):
    inputs = tokenizer(
        examples['document'],
        max_length=512,
        truncation=True,
        padding='max_length',
    )
    outputs = tokenizer(
        examples['summary'],
        max_length=100,
        truncation=True,
        padding='max_length',
    )
    labels = outputs['input_ids']
    labels = [[(label if label != tokenizer.pad_token_id else -100) for label in labels_seq] for labels_seq in labels]
    return {
        'input_ids': inputs['input_ids'],
        'attention_mask': inputs['attention_mask'],
        'labels': labels
    }

train_dataset = train_dataset.map(preprocess_function, batched=True, remove_columns=['document', 'summary'])
validation_dataset = validation_dataset.map(preprocess_function, batched=True, remove_columns=['document', 'summary'])

# --- Load Model ---
model = BartForConditionalGeneration.from_pretrained('facebook/bart-large-cnn')
model.config.label_smoothing_factor = 0.1
model.config.dropout = 0.3
model.config.attention_dropout = 0.2
model.config.decoder_start_token_id = tokenizer.bos_token_id

# --- Training arguments ---
training_args = TrainingArguments(
    output_dir='/content/results_optimized',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    eval_strategy='epoch',  # Evaluate at the end of each epoch
    save_strategy='epoch',  # Save at the end of each epoch
    logging_strategy='epoch',  # Log training loss at the end of each epoch
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model='eval_loss',
    greater_is_better=False,
    learning_rate=2e-5,
    warmup_steps=100,
    gradient_accumulation_steps=8,
    fp16=True,
    weight_decay=0.1,
    gradient_checkpointing=True,
    lr_scheduler_type='cosine',
    report_to='none',
    max_grad_norm=1.0,
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

# --- Train ---
trainer.train()

# --- Save Model and Tokenizer ---
model.save_pretrained('/content/results_optimized/finetuned_bart_large_cnn')
tokenizer.save_pretrained('/content/results_optimized/finetuned_bart_large_cnn')

# --- Extract and Save Metrics ---
metrics = []
for log in trainer.state.log_history:
    epoch = log.get('epoch', None)
    train_loss = log.get('loss', None)
    eval_loss = log.get('eval_loss', None)
    if epoch is not None and (train_loss is not None or eval_loss is not None):
        metrics.append({'epoch': epoch, 'train_loss': train_loss, 'eval_loss': eval_loss})

# Save metrics to CSV
metrics_df = pd.DataFrame(metrics)
metrics_df.to_csv('training_metrics.csv', index=False)
print("\nTraining and Validation Loss Metrics (per epoch):")
print(metrics_df)

print("Training complete and model saved!")

In [ ]:
# %pip install wordcloud
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from transformers import BartTokenizer, BartForConditionalGeneration
from datasets import Dataset
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix, classification_report
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from collections import Counter
import nltk
from matplotlib_venn import venn2
from wordcloud import WordCloud
import uuid

# Set environment variable to avoid distributed training issues
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Create output directory for saving metrics
output_dir = "summarization_metrics"
os.makedirs(output_dir, exist_ok=True)

# Sample data for text summarization
# data = [
#     {
#         'document': (
#             "A massive storm swept through the eastern coast on Friday, causing widespread power outages "
#             "and flooding in several towns. Emergency services responded quickly, evacuating residents "
#             "from high-risk areas. Meteorologists warned that more heavy rainfall is expected over the weekend."
#         ),
#         'summary': (
#             "A storm hit the eastern coast, causing floods and outages, with more rain expected."
#         )
#     }
# ]
# data = [
#     {
#         'document': (
#             "The city council approved a new public transportation initiative on Tuesday, aiming to reduce traffic congestion "
#             "and carbon emissions. The plan includes expanding bus routes, introducing electric buses, and offering discounted "
#             "fares for students and seniors. Officials believe the project will significantly improve urban mobility."
#         ),
#         'summary': (
#             "A new transit plan with electric buses and discounts was approved to ease traffic and cut emissions."
#         )
#     }
# ]
# data = [
#     {
#         'document': (
#             "Lena had always dreamed of becoming a painter, but life in her small village offered few opportunities. "
#             "One day, a traveling art teacher visited the local school and noticed Lena’s sketches. Impressed by her talent, "
#             "he offered to mentor her. Years later, Lena's artwork would be displayed in galleries across the country."
#         ),
#         'summary': (
#             "A village girl’s painting talent is discovered by a mentor, leading her to national art success."
#         )
#     }
# ]
# data = [
#     {
#         'document': (
#             "Regular physical activity has been linked to numerous health benefits, including reduced risk of chronic diseases such as heart disease, diabetes, and obesity." 
#             " Experts recommend at least 150 minutes of moderate exercise per week." 
#             " In addition to physical health, exercise also improves mental well-being by reducing stress and boosting mood."
#         ),
#         'summary': (
#             "Exercise lowers disease risk and boosts mental health; experts advise 150 minutes weekly activity."
#         )
#     }
# ]
data = [
    {
        'document': (
            "Marie Curie was a pioneering physicist and chemist who conducted groundbreaking research on radioactivity. "
            "She was the first woman to win a Nobel Prize and remains the only person to win Nobel Prizes in two different sciences — Physics and Chemistry. "
            "Curie's discoveries led to major advancements in medical treatments, especially in cancer therapy. "
            "Despite working in hazardous conditions, she remained committed to science until her death in 1934 from radiation exposure."
        ),
        'summary': (
            "Marie Curie, a Nobel-winning scientist, made key discoveries in radioactivity that advanced cancer treatment."
        )
    }
]

# Convert sample data to a HuggingFace Dataset
try:
    sample_df = pd.DataFrame(data)
    sample_dataset = Dataset.from_pandas(sample_df)
except Exception as e:
    raise ValueError(f"Error creating dataset: {e}")

# Load the fine-tuned model and tokenizer
model_dir = "/Users/sajinshrestha/Developer/college/text_summarizer/custom-trained-modal-final"
tokenizer = BartTokenizer.from_pretrained(model_dir, local_files_only=True)
model = BartForConditionalGeneration.from_pretrained(model_dir, local_files_only=True)

# Clean and post-process the predicted summary
def clean_summary(text):
    # Split into sentences
    sentences = re.split(r'(?<=[.!?])\s+', text)
    # Remove duplicate sentences and incomplete sentences
    seen = set()
    cleaned = []
    for s in sentences:
        s_strip = s.strip()
        if s_strip and s_strip not in seen and re.match(r'.*[.!?]$', s_strip):
            cleaned.append(s_strip)
            seen.add(s_strip)
    # If last sentence is incomplete, add a period
    if cleaned and not cleaned[-1].endswith('.'):
        cleaned[-1] += '.'
    # Join sentences and normalize whitespace
    result = ' '.join(cleaned)
    result = re.sub(r'\.\.+', '.', result)
    result = re.sub(r'\s+', ' ', result).strip()
    return result

# Perform summarization
input_text = sample_dataset['document'][0]
if not input_text.strip():
    raise ValueError("Input document is empty.")
inputs = tokenizer([input_text], max_length=512, return_tensors='pt', truncation=True, padding=True)
try:
    summary_ids = model.generate(
        inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        num_beams=6,
        max_length=128,
        min_length=40,
        length_penalty=1.0,  
        no_repeat_ngram_size=3,
        early_stopping=True
    )
    predicted_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    predicted_summary = clean_summary(predicted_summary)
except Exception as e:
    raise ValueError(f"Error during summarization: {e}")

actual_summary = sample_dataset["summary"][0]

# Print summaries for reference
print("\n=== Text Summarization Results ===")
print(f"Original Document: {input_text}\n")
print(f"Predicted Summary: {predicted_summary}\n")
print(f"Actual Summary: {actual_summary}\n")
print("="*60)

# Initialize metrics dictionary
metrics = {}

# 1. Summary Length Comparison
pred_len = len(predicted_summary.split())
actual_len = len(actual_summary.split())
plt.figure(figsize=(8, 5))
sns.barplot(x=['Predicted', 'Actual'], y=[pred_len, actual_len], palette='viridis')
plt.title('Summary Length Comparison (Word Count)', fontsize=14)
plt.ylabel('Number of Words', fontsize=12)
plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'summary_length_comparison.png'), dpi=300)
plt.show()
metrics['Length_Predicted'] = pred_len
metrics['Length_Actual'] = actual_len
print("Bar chart comparing the word count of predicted and actual summaries.")
print(f"Description: This chart compares the number of words in the predicted summary ({pred_len}) and the actual summary ({actual_len}). Ideally, the predicted summary should be concise but not too short compared to the reference. If the predicted summary is much shorter, it may miss key information; if much longer, it may be verbose or off-topic.")

# 2. Cosine Similarity (TF-based)
vectorizer = CountVectorizer().fit([predicted_summary, actual_summary])
vectors = vectorizer.transform([predicted_summary, actual_summary]).toarray()
cos_sim = cosine_similarity([vectors[0]], [vectors[1]])[0][0]
plt.figure(figsize=(6, 5))
sns.barplot(x=['Cosine Similarity'], y=[cos_sim], palette='viridis')
plt.title('Cosine Similarity (Predicted vs Actual)', fontsize=14)
plt.ylabel('Similarity Score', fontsize=12)
plt.ylim(0, 1)
plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'cosine_similarity.png'), dpi=300)
plt.show()
metrics['Cosine_Similarity'] = cos_sim
print("Bar chart showing cosine similarity between predicted and actual summaries.")
print(f"Description: Cosine similarity measures the overlap in word usage between summaries. A value of {cos_sim:.3f} (range 0-1) means the summaries are {cos_sim*100:.1f}% similar in terms of word distribution. Values above 0.7 are considered strong; below 0.5 may indicate poor relevance.")

# 3. ROUGE Scores
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
rouge_scores = scorer.score(actual_summary, predicted_summary)
rouge1 = rouge_scores['rouge1'].fmeasure
rouge2 = rouge_scores['rouge2'].fmeasure
rougeL = rouge_scores['rougeL'].fmeasure
plt.figure(figsize=(8, 5))
sns.barplot(x=['ROUGE-1', 'ROUGE-2', 'ROUGE-L'], y=[rouge1, rouge2, rougeL], palette='viridis')
plt.title('ROUGE Scores (F1 Measure)', fontsize=14)
plt.ylabel('F1 Score', fontsize=12)
plt.ylim(0, 1)
plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'rouge_scores.png'), dpi=300)
plt.show()
metrics['ROUGE-1'] = rouge1
metrics['ROUGE-2'] = rouge2
metrics['ROUGE-L'] = rougeL
print("Bar chart showing ROUGE scores (F1) for n-gram overlap between summaries.")
print(f"Description: ROUGE-1 ({rouge1:.3f}) measures unigram overlap, ROUGE-2 ({rouge2:.3f}) measures bigram overlap, and ROUGE-L ({rougeL:.3f}) measures longest common subsequence. High values (above 0.7) indicate the predicted summary captures key phrases and structure from the reference.")

# 4. BLEU Score
reference = [nltk.word_tokenize(actual_summary)]
candidate = nltk.word_tokenize(predicted_summary)
bleu_score = sentence_bleu(reference, candidate, smoothing_function=SmoothingFunction().method1)
plt.figure(figsize=(6, 5))
sns.barplot(x=['BLEU Score'], y=[bleu_score], palette='viridis')
plt.title('BLEU Score', fontsize=14)
plt.ylabel('Score', fontsize=12)
plt.ylim(0, 1)
plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'bleu_score.png'), dpi=300)
plt.show()
metrics['BLEU'] = bleu_score
print("Bar chart showing BLEU score for sequence similarity.")
print(f"Description: BLEU score ({bleu_score:.3f}) evaluates n-gram precision. Scores above 0.5 are good for summarization. Low BLEU may mean the summary uses different wording or order than the reference.")

# 5. Word Overlap Venn Diagram
pred_words = set(predicted_summary.lower().split())
actual_words = set(actual_summary.lower().split())
plt.figure(figsize=(6, 6))
venn2([actual_words, pred_words], set_labels=('Actual', 'Predicted'), set_colors=('#1f77b4', '#ff7f0e'))
plt.title('Word Overlap Venn Diagram', fontsize=14)
plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'word_overlap_venn.png'), dpi=300)
plt.show()
overlap = len(pred_words & actual_words)
total = len(actual_words)
metrics['Word_Overlap_Ratio'] = overlap / total if total > 0 else 0
print("Venn diagram visualizing word overlap between predicted and actual summaries.")
print(f"Description: {overlap} words overlap out of {total} actual summary words. High overlap means the summary is relevant, but excessive overlap may mean copying instead of abstraction. A good summary balances overlap and novelty.")

# 6. Pie Chart of Overlap vs Non-Overlap
non_overlap = len((pred_words | actual_words) - (pred_words & actual_words))
plt.figure(figsize=(6, 6))
plt.pie([overlap, non_overlap], labels=['Overlap', 'Non-Overlap'], autopct='%1.1f%%', colors=['#1f77b4', '#ff7f0e'])
plt.title('Word Overlap vs Non-Overlap', fontsize=14)
plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'overlap_pie_chart.png'), dpi=300)
plt.show()
print("Pie chart showing proportion of overlapping and non-overlapping words.")
print(f"Description: {overlap/(overlap+non_overlap):.2%} of words overlap. A balanced ratio (40-70%) is ideal. Too high means copying, too low means poor relevance.")

# 7. Word Frequency Distribution
all_words = nltk.word_tokenize(predicted_summary + ' ' + actual_summary)
word_freq = Counter(all_words)
most_common = word_freq.most_common(10)
words, freqs = zip(*most_common) if most_common else ([], [])
if words:
    plt.figure(figsize=(10, 5))
    sns.barplot(x=list(words), y=list(freqs), palette='viridis')
    plt.title('Top 10 Word Frequency Distribution', fontsize=14)
    plt.ylabel('Frequency', fontsize=12)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'word_frequency.png'), dpi=300)
    plt.show()
print("Bar chart showing the most frequent words in both summaries.")
print(f"Description: Frequent words ({', '.join(words)}) should be key concepts. If stopwords dominate, the summary may lack substance.")

# 8. Word Cloud for Predicted and Actual Summaries
combined_text = predicted_summary + ' ' + actual_summary
wordcloud = WordCloud(width=800, height=400, background_color='white', colormap='viridis').generate(combined_text)
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud of Predicted and Actual Summaries', fontsize=14)
plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'word_cloud.png'), dpi=300)
plt.show()
print("Word cloud visualizing the most prominent words in both summaries.")
print("Description: Larger words are more frequent. Good summaries highlight key concepts, not just repeated words. If irrelevant words are large, the summary may be off-topic.")

# 9. Token-level Precision, Recall, F1, Accuracy
pred_tokens = predicted_summary.split()
actual_tokens = actual_summary.split()
all_tokens = list(set(pred_tokens + actual_tokens))
pred_bin = [1 if t in pred_tokens else 0 for t in all_tokens]
actual_bin = [1 if t in actual_tokens else 0 for t in all_tokens]
precision = precision_score(actual_bin, pred_bin)
recall = recall_score(actual_bin, pred_bin)
f1 = f1_score(actual_bin, pred_bin)
accuracy = accuracy_score(actual_bin, pred_bin)
plt.figure(figsize=(8, 5))
sns.barplot(x=['Precision', 'Recall', 'F1', 'Accuracy'], y=[precision, recall, f1, accuracy], palette='viridis')
plt.title('Token-level Precision, Recall, F1, Accuracy', fontsize=14)
plt.ylim(0, 1)
plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'token_metrics.png'), dpi=300)
plt.show()
metrics['Precision'] = precision
metrics['Recall'] = recall
metrics['F1'] = f1
metrics['Accuracy'] = accuracy
print("Bar chart showing token-level precision, recall, F1, and accuracy.")
print(f"Description: Precision={precision:.3f}, Recall={recall:.3f}, F1={f1:.3f}, Accuracy={accuracy:.3f}. Values above 0.7 are good. Low values mean the model misses or mispredicts key tokens. Precision is the fraction of predicted tokens that are correct; recall is the fraction of actual tokens that are found; F1 balances both.")

# 10. Confusion Matrix
cm = confusion_matrix(actual_bin, pred_bin)
plt.figure(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Not in Pred', 'In Pred'], yticklabels=['Not in Actual', 'In Actual'])
plt.title('Token-level Confusion Matrix', fontsize=14)
plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'confusion_matrix.png'), dpi=300)
plt.show()
print("Heatmap showing the confusion matrix for token-level prediction.")
print(f"Description: The confusion matrix shows true positives (bottom-right), true negatives (top-left), false positives (top-right), and false negatives (bottom-left). High true positives/negatives are good; high false values mean the model is missing or mispredicting tokens.")

# 11. Sentence Length Difference (in tokens)
pred_sent_len = [len(nltk.word_tokenize(sent)) for sent in nltk.sent_tokenize(predicted_summary)]
actual_sent_len = [len(nltk.word_tokenize(sent)) for sent in nltk.sent_tokenize(actual_summary)]
plt.figure(figsize=(8, 5))
sns.boxplot(data=[pred_sent_len, actual_sent_len], palette='viridis')
plt.xticks([0, 1], ['Predicted', 'Actual'])
plt.title('Sentence Length Distribution (Tokens)', fontsize=14)
plt.ylabel('Tokens per Sentence', fontsize=12)
plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'sentence_length_boxplot.png'), dpi=300)
plt.show()
print("Boxplot comparing sentence length distribution in predicted and actual summaries.")
print(f"Description: Predicted sentences have lengths: {pred_sent_len}. Actual: {actual_sent_len}. Ideally, sentence lengths should be similar, indicating comparable structure. Large differences may mean poor coherence or structure.")

# 12. Metrics Summary Table
metrics_table = pd.DataFrame({
    'Metric': list(metrics.keys()),
    'Score': list(metrics.values())
}).round(3)
print("\n=== Metrics Summary ===")
print(metrics_table)
metrics_table.to_csv(os.path.join(output_dir, 'metrics_summary.csv'), index=False)
print("Table summarizing all calculated metrics.")
print("Description: This table aggregates all metrics for a holistic view. High scores across metrics indicate a strong summarizer. Review for consistency and outliers.")

# 13. Classification Report
print("\n=== Token-level Classification Report ===")
print(classification_report(actual_bin, pred_bin, target_names=['Not in Summary', 'In Summary']))
print("Detailed classification report for token-level prediction. High precision and recall are desired. Look for balanced support and low false positives/negatives.")

# --- Status Analysis ---
print("\n=== Status Analysis ===")
if cos_sim > 0.7 and rouge1 > 0.7 and bleu_score > 0.5 and f1 > 0.7:
    print("Status: The summarization model is performing well. Most metrics are in the good range. Summaries are relevant, concise, and well-structured. No major issues detected.")
elif cos_sim > 0.5 and rouge1 > 0.5 and f1 > 0.5:
    print("Status: The model is moderately effective. Some metrics are below optimal. Consider tuning model parameters, increasing training data, or improving preprocessing for better abstraction and token prediction.")
else:
    print("Status: The model needs improvement. Metrics are below recommended thresholds. Consider more training, data cleaning, or model architecture changes. Review individual diagrams for specific weaknesses.")